# Handing Secrets

### What is a Secret?

Example of the secret:
- Random token e.g. `http://localhost:8888/?token=a67d2658ed1551eb5cdeca901762760060c1040542f8f6ae`
- Password

### How to Protect it?

Require to be protected over it's lifetime:
1. Generating
2. Storing
3. Verifying
4. Retaining 

> Use cryptography functions, otherwise your secrets can leak

### Don't Rool Your Own Crypto

- You can rool your own, but probably make security mistakes
- Prefer open source, widely known tools and libraries 

You can roll your own, but you probably will make a major security mistake if you are not an expert in security/cryptography or have had your scheme analyzed by multiple experts. I'm more willing to bet on an open-source publicly known encryption scheme that's out there for all to see and analyze. More eyes means more likely that the current version doesn't have major vulnerabilities, as opposed to something developed in-house by non-experts.

From Phil Zimmermann's (PGP creator) Introduction to Cryptography (Page 54):

When I was in college in the early 70s, I devised what I believed was a brilliant encryption scheme. A simple pseudorandom number stream was added to the plaintext stream to create ciphertext. This would seemingly thwart any frequency analysis of the ciphertext, and would be uncrackable even to the most resourceful government intelligence agencies. I felt so smug about my achievement.

Years later, I discovered this same scheme in several introductory cryptography texts and tutorial papers. How nice. Other cryptographers had thought of the same scheme. Unfortunately, the scheme was presented as a simple homework assignment on how to use elementary cryptanalytic techniques to trivially crack it. So much for my brilliant scheme.

From this humbling experience I learned how easy it is to fall into a false sense of security when devising an encryption algorithm. Most people don’t realize how fiendishly difficult it is to devise an encryption algorithm that can withstand a prolonged and determined attack by a resourceful opponent.

# Random Numbers


<img alt="" src="data:image/jpeg;base64, ..." />

<sub><sup>By Dean Hochman from Overland Park, Kansas, U.S. - lava lamps, CC BY 2.0, https://commons.wikimedia.org/w/index.php?curid=79022567</sup></sub>

## Where Random Numbers are Used?

- Authentication, web session protection
- Password reset and generation

## Common Pitfails

Not all random numbers are cryptographycally secure

In [16]:
import random
random.seed(2)
print(random.random())
print(random.random())

0.9560342718892494
0.9478274870593494


## The `secrets` Module

For cryptography purposes
use [secrets](https://docs.python.org/3/library/secrets.html) module.

In [26]:
import secrets
secrets.token_bytes()

b'\xb7\x96{mA0\xda\xae\xc5\x989\xcbB/\xc8\xca\\\x1c\x00\xc2j\x07v\xc74\xe4\x91\n7+\x02\xc7'

> A random __token size__ is not part of API contract - it can change any time
even with module update.

## Secure Randoms should be Long 

How big shoudl be the secret to protect from
[brute-force](https://en.wikipedia.org/wiki/Brute-force_attack)?

In [33]:
import secrets
secrets.token_bytes(nbytes=32)

b'\xd5E\xe4\x8f1F\x90\x11Xt\xc3\xdf?v\xc9w\x91^\x92\xf3j\xae\xf7\x04\xcfh\x12m\x18$\x08^'

### How to Deal with URLs?

The `secrets.token_hex()` output all ASCII characters:

In [38]:
import secrets
secrets.token_hex()

'560234df217781a05c7e96a5380464d6a64a90f35d6a6a836b9b1a717fb30749'

And `secrets.token_urlsafe()` gives even shorter string:

In [42]:
import secrets
secrets.token_urlsafe()

'6DbXaKmMKm_18x2sFU75LKIDAW8k0jn2FSbi3PspZDg'

## Comparing Secret Tokens

### Naive Secrets Comparison

In [43]:
import secrets
ADMIN_TOKEN = secrets.token_urlsafe()

def is_authorized(token):
    return token = = ADMIN_TOKEN

some_token = secrets.token_urlsafe()
is_authorized(some_token)

False

Checking if a token you've just received matches one you have generated before
looks stright forward, but only on first sight. The operations with a secrets
need to be resistant to [timing
attacks](https://codahale.com/a-lesson-in-timing-attacks/).

> "Naive Comparison" is vulnerable to a [timing
attack](https://codahale.com/a-lesson-in-timing-attacks/).

### Timing Attack

In [46]:
import secrets
ADMIN_TOKEN = secrets.token_urlsafe()

def is_authorized(token):
    for i in range(len(token)):
        if token[i] != ADMIN_TOKEN[i]:
            return False
    return True

some_token = secrets.token_urlsafe()
import timeit

print(timeit.Timer(lambda: is_authorized(some_token)).timeit(100000))
print(timeit.Timer(lambda: is_authorized(ADMIN_TOKEN)).timeit(100000))

0.06687529999908293
0.5788790999977209


### TODO: [Lab on timing attack with partial code](https://sqreen.github.io/DevelopersSecurityBestPractices/timing-attack/python)

### Secure Secrets Comparison

The `secrets.compare_digest(a, b)` do a comparison in a way to reduce a risk of timig attacks:

In [8]:
import secrets
ADMIN_TOKEN = secrets.token_urlsafe()

def is_authorized(token):
    return secrets.compare_digest(token, ADMIN_TOKEN)

some_token = secrets.token_urlsafe()
is_authorized(some_token)

False

### Mitigating Timing Attack

Constant time comparison mitigates timing attack

In [57]:
import secrets
ADMIN_TOKEN = secrets.token_urlsafe()

def is_authorized(token):
    return secrets.compare_digest(token, ADMIN_TOKEN)

some_token = secrets.token_urlsafe()
import timeit

print(timeit.Timer(lambda: is_authorized(some_token)).timeit(100000))
print(timeit.Timer(lambda: is_authorized(ADMIN_TOKEN)).timeit(100000))

0.03308619999734219
0.03506039999774657


## Passwords Management

### DO NOT Store a Password

> Never store passwords or other secrets in a recoverable format whether it plain
or encrypted.

### How to Store a Password? 

- Hash passwords before storing
- Use special Password Hashing algorithms
- Choose secure parameters for password hashing

In [61]:
import hashlib, secrets
salt = secrets.token_bytes(nbytes=16)
hashlib.pbkdf2_hmac(hash_name='sha256',
                    password=b'secretpassword',
                    salt=salt,
                    iterations=10000)

b'\x03\xa5\xf2\x10\xab\xda\x10\xbb2\x17Y\xbc\xbb\x95\xc2\x1d\xe1\xd9\x7fuyI?X\xeb\x87\xc4\xa4d\x19Q\x1a'

When storing a passwords think about protecting your storage from brute force
attacks. The first consideration is that since brute force attacks require many
iterations of guessing a password the password check shoudl be slow enough. And
the second consideraion is that similar passwords shoud not look the same.

https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html#work-factors
PBKDF2 is recommended by NIST and has FIPS-140 validated implementations. So, it should be the preferred algorithm when these are required. Additionally, it is supported out of the box in the .NET framework, so is commonly used in ASP.NET applications.

PBKDF2 can be used with HMACs based on a number of different hashing algorithms. HMAC-SHA-256 is widely supported and is recommended by NIST.

The work factor for PBKDF2 is implemented through the iteration count, which should be at least 10,000 (although values of up to 100,000 may be appropriate in higher security environments).

### Hashing vs. Encryption

Hash is a on-way function <= impossible to recover a password.

Encryption is a two-way function <= password can be recovered.

**Ability to decrypt a password poses a serious security risk.**

https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html#hashing-vs-encryption
In almost all circumstances, passwords should be hashed rather than encrypted, as this makes it difficult or impossible for an attacker to obtain the original passwords from the hashes.

Encryption should only be used in edge cases where it is necessary to be able to obtain the original password. Some examples of where this might be necessary are:

If the application needs to use the password to authenticate against an external legacy system that doesn't support SSO.
If it is necessary to retrieve individual characters from the password.

### Choosing Parameters: Salting

- Salt is a random number, unique per each stored credential
- Salt is stored along with password hash
- Salting makes hashes for the same passwords different

> The salt should be at least 16 bytes

https://auth0.com/blog/adding-salt-to-hashing-a-better-way-to-store-passwords/#Attacking-Unsalted-Passwords
To start, the attacker could try a dictionary attack. Using a pre-arranged listing of words, such as the entries from the English dictionary, with their computed hash, the attacker easily compares the hashes from a stolen passwords table with every hash on the list. If a match is found, the password then can be deduced.

https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html#salting
Modern hashing algorithms such as Argon2 or Bcrypt automatically salt the passwords, so no additional steps are required when using them. However, if you are using a legacy password hashing algorithm then salting needs to be implemented manually. The basic steps to perform this are:
Generate a salt using a cryptographically secure function.
The salt should be at least 16 characters long.
Encode the salt into a safe character set such as hexadecimal or Base64.
Combine the salt with the password.
This can be done using simple concatenation, or a construct such as a HMAC.
Hash the combined password and salt.
Store the salt and the password hash.

### Choosing Parameters: Iterations / Work factor

- iterations := 2 ^ work factor
- More iteration >>> longer computation >>> brute force mitigated
- Given Mores's law the work factor need to be incremented every 18 months

> The PBKDF2 iterations should be at least 10000 (or 100000 for higher security requirements)

The work factor is essentially the number of iterations of the hashing algorithm that are performed for each password (usually it's actually 2^work iterations). The purpose of the work factor is to make calculating the hash more computationally expensive, which in turn reduces the speed at which an attacker can attempt to crack the password hash. The work factor is typically stored in the hash output.

When choosing a work factor, a balance needs to be struck between security and performance. Higher work factors will make the hashes more difficult for an attacker to crack, but will also make the process of verifying a login attempt slower. If the work factor is too high, this may degrade the performance of the application, and could also be used by an attacker to carry out a denial of service attack by making a large number of login attempts to exhaust the server's CPU.

There is no golden rule for the ideal work factor - it will depend on the performance of the server and the number of users on the application. Determining the optimal work factor will require experimentation on the specific server(s) used by the application. As a general rule, calculating a hash should take less than one second, although on higher traffic sites it should be significantly less than this.

Upgrading the Work Factor¶
One key advantage of having a work factor is that it can be increased over time as hardware becomes more powerful and cheaper. Taking Moore's Law (i.e, that computational power at a given price point doubles every eighteen months) as a rough approximation, this means that the work factor should be increased by 1 every eighteen months.

The most common approach to upgrading the work factor is to wait until the user next authenticates, and then to re-hash their password with the new work factor. This means that different hashes will have different work factors, and may result in hashes never being upgraded if the user doesn't log back in to the application. Depending on the application, it may be appropriate to remove the older password hashes and require users to reset their passwords next time they need to login, in order to avoid storing older and less secure hashes.

### Useful Links

- https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html - OWASP Password Storage Cheat Sheet
- https://goto.intel.com/cryptoguidelines - Intel Crypto Guidelines